# EM CONSTRUÇÃO

# CHALLENGE DADOS

Neste challenger o desafio com esses dados, tentar modelar um classificador capaz de encontrar potenciais clientes inadimplentes e solucionar o problema do Alura Cash.

### OS DADOS

Foi disponibilizado um arquivo DUMP que fiz os tratamentos necessários, unir as tabelas e exportei como um .csv

DICIONÁRIO DE DADOS:

* **pessoa idade:** Idade da pessoa em anos que solicia empréstimo.   
* **salario_ano:** Salário anual da pessoa solicitante.
* **propriedade_sit:** Situação da propriedade que a pessoa possui: Alugada, Própria, Hipotecada e Outros.
* **ano_trabalhado:** Tempo em anos que a pessoa trabalhou
* **motivo_emprestimo:** Motivo do empréstimo: Pagamento de debitos, Pessoal, Educativo, Medico, Empreendimento, Melhora do lar
* **pontuacao_emprestimos:** Pontuação de empréstimos, por nível variando de A a G
* **vl_total:** Valor total do empréstimo solicitado
* **tx_juros:** Taxa de juros
* **inadimplencia:** Possibilidade de inadimplência
* **tx_renda_divida:** Renda percentual entre o valor total do empréstimo e o salário anual
* **hst_inadimplencia:** Indica se a pessoa já foi inadimplente 1 SIM, 0 NÃO
* **hst_primeiro_credito:** Tempo - em anos - desde a primeira solicitação de crédito ou aquisição de um cartão de crédito

## IMPORTAÇÕES DE BIBLIOTECAS

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## IMPORTAÇÃO DO DATASET

In [7]:
dados = pd.read_csv('dados_juntos.csv', sep=';')
dados.head()

,pessoa_idade,salario_ano,propriedade_sit,ano_trabalhado,motivo_emprestimo,pontuacao_emprestimos,vl_total,tx_juros,inadimplencia,tx_renda_divida,hst_inadimplencia,hst_primeiro_credito
0,23,108000,Hipotecada,5,Pagamento de debitos,D,10000,0.00,1.0,0.09,1.0,2
1,31,35000,Hipotecada,2,Pagamento de debitos,B,10225,9.99,0.0,0.29,0.0,6
2,22,14400,Propria,0,Pessoal,B,2100,11.14,0.0,0.15,0.0,4
3,21,47580,Hipotecada,5,Educativo,A,14000,6.03,0.0,0.29,0.0,2
4,33,115000,Hipotecada,10,Medico,B,25000,11.86,0.0,0.22,0.0,8


In [9]:
print(f'O dataset tem {dados.shape[0]} Linhas e {dados.shape[1]} Colunas')

O dataset tem 34485 Linhas e 12 Colunas
